In [1]:
import chromadb
import torch
import re
import os


from chromadb import PersistentClient



from transformers import (
    AutoModel, AutoTokenizer,AutoModelForCausalLM,TrainingArguments, DataCollatorForLanguageModeling, Trainer
)

from week5.tools.get_embedding import get_embedding

from datasets import load_dataset

from peft import PeftModel

In [2]:
qwen_model_path = '/home/wby/projects/model/Qwen-7B-Chat'
db_path = '/home/wby/projects/week5/chroma_db/chroma_recipe_db'
model_path = '/home/wby/projects/model/all-MiniLM-L6-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
#连接数据库
chroma_client = PersistentClient(
    path=db_path,
    settings=chromadb.config.Settings(
        anonymized_telemetry=False,
        allow_reset=False
    )
)

In [4]:
#查看我的数据库里有多少集合
collections = chroma_client.list_collections()
print(f"数据库里共有{len(collections)}个集合")
for coll in collections:
    print(f"集合名称{coll.name}")

数据库里共有1个集合
集合名称recipe_50w_384d_ver2


In [5]:
#获取集合，连接模型是使用rag的基础
collection = chroma_client.get_collection(name='recipe_50w_384d_ver2')
model = AutoModel.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code=True) #注意！allMiniLM-V6的分词器

In [6]:
def parse_conditions(query):
    """
    从用户查询中提取条件，比如calories < 200  time > 10 转为Chroma支持的过滤格式
    :param query: 询问的参数
    :return: Chroma过滤字典（如{"Calories": {"$lt": 200}, "TotalTime": {"$gt": 10}}）
    """
    field_mapping = {
        # 键：用户可能输入的关键词（小写）；值：元数据中对应的字段名
        "calorie": "Calories",
        "calories": "Calories",
        "kcal": "Calories",
        "time": "TotalTime",
        "totaltime": "TotalTime",
        "duration": "TotalTime",
        "serving": "RecipeServings",
        "servings": "RecipeServings",
        "portion": "RecipeServings"
    }

    # 2. 定义运算符映射：用户输入的符号 → Chroma支持的运算符
    operator_mapping = {
        "<": "$lt",
        "<=": "$lte",
        ">": "$gt",
        ">=": "$gte",
        "=": "$eq",
        "==": "$eq",
        'less than':"$lt",
        'greater than':"$gt",
        'less than or equal to':"$lte",
        'greater than or equal to':"$gte",
        'equal to':"$eq",
    }

    # 3. 用正则表达式提取所有条件（支持多条件，用and连接）
    # 匹配格式：[关键词] [运算符] [数字]（如 "calories < 200"、"time >= 10"）
    pattern = r"(\w+\s*\w*)\s*([<>=]{1,2})\s*(\d+\.?\d*)"
    matches = re.findall(pattern, query.lower())  # 转小写，忽略大小写

    #4. 处理提取到的条件，转换为Chroma格式
    conditions = []
    for match in matches:
        # match = (用户输入关键词, 运算符, 数值)
        user_filed, user_op, value = match

        #首先去除关键词中的空格
        # 去除关键词中的空格（如"total time" → "totaltime"）
        user_filed_clean= user_filed.replace(" ","")
        meta_field = None
        for key in field_mapping:
            if key in user_filed_clean:
                meta_field = field_mapping[key]
                break

        if not meta_field:
            continue #不认识的字段直接跳过

         # 4.2 转换运算符（如"<" → "$lt"）
        chroma_op = operator_mapping.get(user_op,None)
        if not chroma_op:
            continue # 不支持的运算符， 跳过

        try:
            value = float(value)
        except:
            continue  #数值无效，跳过

        conditions.append({meta_field: {chroma_op: value}})

    if len(conditions) == 0:
         return None # 没有条件， 后记组合时chroma会过滤的
    elif len(conditions) == 1:
        return conditions[0] # 单个条件直接返回
    else:
        return {'$and':conditions}



In [7]:
query1 = "Find Low-Fat Dessert with calories < 200"
print("测试1结果：", parse_conditions(query1))
# 预期输出：{"Calories": {"$lt": 200}}

# 测试用例2：多个条件（耗时>10且<30，份量=2）
query2 = "Recipe with total time >10 and total time <30, servings = 2"
print("测试2结果：", parse_conditions(query2))
# 预期输出：{"$and": [{"TotalTime": {"$gt": 10}}, {"TotalTime": {"$lt": 30}}, {"RecipeServings": {"$eq": 2}}]}

 # 测试用例3：带小数的条件（热量>=150.5）
query3 = "kcal >= 150.5"
print("测试3结果：", parse_conditions(query3))
# 预期输出：{"Calories": {"$gte": 150.5}}

# 测试用例4：无有效条件
query4 = "Find Banana Flambe recipe"
print("测试4结果：", parse_conditions(query4))

测试1结果： {'Calories': {'$lt': 200.0}}
测试2结果： {'$and': [{'TotalTime': {'$gt': 10.0}}, {'TotalTime': {'$lt': 30.0}}, {'RecipeServings': {'$eq': 2.0}}]}
测试3结果： {'Calories': {'$gte': 150.5}}
测试4结果： None


In [8]:
"""query_embedding = get_embedding('quick banana flambe',tokenizer, model,device)"""

def rag_retrieve(query,top_k=3):
    """
    基础的rag函数，根据查询返回最最相似的食谱
    :param query: 用户查询的问题
    :param top_k: 返回前k个最相近的结果
    :return 返回包括食谱信息的列表|
    """
    query_emb = get_embedding(query,tokenizer, model,device)

    #添加条件
    filter_conditions = parse_conditions(query)

    resluts = collection.query(
        query_embeddings = [query_emb],
        n_results=top_k,
        include=['metadatas','documents','distances'],
        where= filter_conditions
    )
    """print(f"这是一个测试ids{resluts['ids']}")
    print(f"这是一个测试ids{resluts['distances']}")
    print(f"这是一个测试ids{resluts['metadatas']}")
    print(f"这是一个测试documents{resluts['documents']}")"""
    formatted_resluts = []
    for i,(doc_id,distance,meta, doc_text) in enumerate(zip(
        resluts['ids'][0], resluts['distances'][0],
        resluts['metadatas'][0], resluts['documents'][0]
    )):
        formatted_resluts.append({
            'doc_id': doc_id,
            'distance': distance,
            'meta': meta,
            'text': doc_text
        })
    return formatted_resluts



In [9]:

def load_qwen_tokenizer(qwen_model_path):
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            qwen_model_path,
            trust_remote_code = True
        )
        qwen_official_chat_template = """
        {% for message in messages %}
            {% if message['role'] == 'user' %}
                <|im_start|>user
                {{ message['content'] }}
                <|im_end|>
            {% elif message['role'] == 'assistant' %}
                <|im_start|>assistant
                {{ message['content'] }}
                <|im_end|>
            {% endif %}
        {% endfor %}
        {% if add_generation_prompt %}
            <|im_start|>assistant
        {% endif %}
        """
        tokenizer.chat_template = qwen_official_chat_template  # 把模板赋值给分词器
        tokenizer.pad_token = tokenizer.eos_token  # 用结束符作为填充符（官方推荐）
        print('Qwen 分词器加载成功！')
        return tokenizer
    except Exception as e:
        print(e)
        return None

qwen_tokenizer = load_qwen_tokenizer(qwen_model_path)

Qwen 分词器加载成功！


In [10]:

def load_qwen_model(qwen_model_path):
    try:
        print("开始加载模型，模型较大，耐心等待，祈祷没人占用GPU")
        model = AutoModelForCausalLM.from_pretrained(
            qwen_model_path,
            device_map = 'auto',
            trust_remote_code = True,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True,
        ).eval()
        print('Qwen-7B-Chat 模型加载成功')
        return model

    except Exception as e:
        error_msg = str(e)
        if "out of memory" in error_msg.lower():
            print(f"❌ 显存不足！解决方案：1. 改用 CPU 加载（把 device_map 设为 'cpu'）；2. 用 Int4 量化模型（需装 auto-gptq）")
        elif "No such file or directory" in error_msg:
            print(f"❌ 模型路径错误！请确认 {model_path} 下有 config.json、pytorch_model-*.bin 等文件")
        else:
            print(f"❌ 模型加载失败：{error_msg}")
        return None

In [11]:
if qwen_tokenizer:
    base_model = load_qwen_model(qwen_model_path)
    qwen_model = PeftModel.from_pretrained(base_model, "/home/wby/projects/week5/data/qwen_food_lora/final_lora")
    #qwen_model = load_qwen_model(qwen_model_path)
else:
    print('分词器未加载，无法加载模型')

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


开始加载模型，模型较大，耐心等待，祈祷没人占用GPU


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Qwen-7B-Chat 模型加载成功


In [12]:
def build_recipe_prompt(query, retrieved_results, prompt_template):
    """"

    """
    retrieved_str = ""
    for i,result in enumerate(retrieved_results,1):
        doc_id = result['doc_id']
        distance = round(result['distance'],1)
        recipe_name = result['meta']['Name']
        total_time = round(result['meta']['TotalTime'],1)
        doc_text = result['text']
        calories = round(result['meta']['Calories'],1)
        category = result['meta']['RecipeCategory']

        #拼接单条数据
        retrieved_str += f"""
        第{i}条数据结果：
        - 文档ID: {doc_id}
        - 相似度: {distance}
        -  名称: {recipe_name}
        -  种类: {category}
        - 总时间: {total_time} min
        -  热量: {calories} 卡
        -食谱核心: {doc_text}
        """

    final_template = prompt_template.replace("{{query}}", query).replace("{{retrieved_results}}", retrieved_str)
    return final_template


In [13]:
# 1. 核心搭配提取（不变，处理带动作的query）
def extract_core_pair(query):
    food_actions = ["eat", "cook","cooking", "make", "bake", "fry", "prepare", "taste"]
    # 1. 先清理特殊符号（问号、小于号等）
    query_clean = re.sub(r'[?.,!;<>≤≥]', '', query).strip().lower()
    # 2. 过滤“时间/热量”等非食物条件词（关键新增）
    condition_words = ["time", "min", "minute", "hour", "calorie", "kcal", "cal"]
    for word in condition_words:
        query_clean = re.sub(rf'\b{word}\b\s*[0-9]*', '', query_clean).strip()

    # 3. 提取“动作+食物”核心搭配
    for action in food_actions:
        pattern = rf'\b{action}\b\s*(.*)'
        match = re.search(pattern, query_clean)
        if match:
            core_object = match.group(1).strip()
            stop_words = ["some", "the", "a", "an", "my", "your"]
            for stop in stop_words:
                core_object = re.sub(rf'\b{stop}\b', '', core_object).strip()
            if core_object:
                core_pair = f"{action} {core_object}"
                print(f"🎯 提取核心搭配：[{query}] → [{core_pair}]")
                return core_pair
    return query_clean

# 2. 单字词补全（不变）
def complete_single_word_query(query):
    if len(query.strip().split()) == 1:
        completed_query = f"eat {query.strip()}"
        return completed_query
    if len(query.strip().split()) == 2:
        completed_query = f"how to make {query.strip()}"
        return completed_query
    return query

# 3. 最终判断函数（新增“纯食物名词”规则）
def is_food_query_final_solution(query, model, tokenizer):
    """
    判定一段对话是否是和讨论美食相关的话题
    """
    core_query = extract_core_pair(query)
    completed_query = complete_single_word_query(core_query)

    # 关键新增：规则同时包含“动作+食材”和“纯食物名词组合”
    judge_prompt = f"""
    用户类型： 那些讨论美食相关话题的用户
    规则：1. 满足以下任一条件→YES，否则NO，仅输出YES/NO。
         2. 判定用户话题如果是与食物相关的->YES, 否则NO, 仅输出YES/NO
    1. 含eat/cook/cooking/make/bake/fry/about+食材（如make banana dessert→YES）；
    2. 纯食物名词组合.
    示例1：eat apple→YES；示例2：banana dessert→YES；示例3：use phone→NO。
    现在判断：{completed_query}→
    """


    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [
                {'role':"system", "content": '你是一个专业的吃货，只需判读这段对话是不是在讨论与美食相关的话题'},
                {"role": "user", "content": judge_prompt},
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        outputs = model.generate(
            inputs,
            max_new_tokens=3,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            attention_mask=torch.ones_like(inputs)
        )

    raw_output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).strip()
    clean_output = re.sub(r'[。.\s]', '', raw_output).upper()
    print(f"🔍 是否是食物核心搭配判断：[{completed_query}] → 输出：[{raw_output}]")
    return clean_output

In [14]:
# 测试
query1 = "Spicy Chicken Noodles"
result1 = is_food_query_final_solution(query1, qwen_model, qwen_tokenizer)
print(f"测试1（{query1}）返回：{result1}\n")  # 预期：YES

query2 = "apple pie"
result2 = is_food_query_final_solution(query2, qwen_model, qwen_tokenizer)
print(f"测试2（{query2}）返回：{result2}\n")  # 预期：NO

query3 = "milk"
result3 = is_food_query_final_solution(query3, qwen_model, qwen_tokenizer)
print(f"测试3（{query3}）返回：{result3}\n")  # 预期：YES

query4 = "Kung Pao chicken"
result4 = is_food_query_final_solution(query4, qwen_model, qwen_tokenizer)
print(f"测试4（{query4}）返回：{result4}\n")  # 预期：YES

query5 = "yu xiang shredded pork"
result5 = is_food_query_final_solution(query5, qwen_model, qwen_tokenizer)
print(f"测试5（{query5}）返回：{result5}\n")  # 预期：NO

/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:612: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


🔍 是否是食物核心搭配判断：[spicy chicken noodles] → 输出：[YES]
测试1（Spicy Chicken Noodles）返回：YES

🔍 是否是食物核心搭配判断：[how to make apple pie] → 输出：[YES
判断]
测试2（apple pie）返回：YES判断

🔍 是否是食物核心搭配判断：[eat milk] → 输出：[YES]
测试3（milk）返回：YES

🔍 是否是食物核心搭配判断：[kung pao chicken] → 输出：[YES]
测试4（Kung Pao chicken）返回：YES

🔍 是否是食物核心搭配判断：[yu xiang shredded pork] → 输出：[YES]
测试5（yu xiang shredded pork）返回：YES



In [15]:
# 验证适配器是否加载成功
def check_lora_loaded(model):
    # 检查模型是否包含LoRA适配器参数
    has_lora = any("lora" in name for name, _ in model.named_parameters())
    print(f"LoRA适配器是否加载：{'✅ 已加载' if has_lora else '❌ 未加载'}")
    # 检查模型精度
    dtype = next(model.parameters()).dtype
    print(f"模型精度：{dtype}（应为torch.float16）")

# 在测试代码中加入验证
check_lora_loaded(qwen_model)  # 替换成你的模型变量名

LoRA适配器是否加载：✅ 已加载
模型精度：torch.bfloat16（应为torch.float16）


In [16]:
# 提取一句话中的所有食物相关的单词
def is_food_list(query, qwen_model, qwen_tokenizer):
    """
    基于 QWen-7B 模型，提取查询中的食物名词列表
    :param query: 用户查询（字符串）
    :param qwen_model: 加载好的 QWen-7B 模型
    :param qwen_tokenizer: 加载好的 QWen 分词器
    :return: 食物名词列表（如 ['apple pie']）
    """
    core_query = extract_core_pair(query)
    # 1. 构造提示词模板（替换 {{query}} 为实际用户查询）
    prompt_template = f"""###任务：基于用户的询问，提取话语中的与食物相关的名词（只能提取食物名词）
    用户查询（query）：{core_query}

    ### 示例：
    1. query = 'i want to cook french fries' 输出：['french fries']
    2. query = 'i want to cook apple pie and orange' 输出：['apple pie' , 'orange']
    3. query = 'i want to make apple pie and Trump' 输出：['apple pie']

    ### 注意事项:
    1. 你只能提取与食物有关的单词。
    2. 不能提取与食物无关的单词（例如人名、地名）

    ###输出格式
    请按照以下格式输出：
    ['food1','food2','food3']"""

    # 填充用户查询到提示词中
    #prompt = prompt_template.format(query=query)

    # 2. 用 QWen 分词器处理提示词（适配 QWen 聊天格式）
    inputs = qwen_tokenizer.apply_chat_template(
        [
            {'role':"system", "content": '你是一个专业的吃货，需要从用户的一段话里提取出所有与食物有关的单词'},
            {"role": "user", "content": prompt_template}
        ],  # 按 QWen 要求的聊天格式组织
        add_generation_prompt=True,  # 自动添加模型回复的前缀
        return_tensors="pt"  # 返回 PyTorch 张量
    ).to(qwen_model.device)  # 移到模型所在设备（GPU/CPU）

    # 3. 模型生成结果（控制输出长度，避免冗余）
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = qwen_model.generate(
            inputs,
            max_new_tokens=50,  # 食物列表长度有限，50个token足够
            temperature=0.1,  # 降低随机性，确保输出格式稳定
            top_p=0.9,
            do_sample=False,  # 确定性生成，避免重复结果
            eos_token_id=qwen_tokenizer.eod_id  # 遇到结束符停止生成
        )

    # 4. 解码输出并提取食物列表
    # 提取模型生成的部分（排除输入提示词）
    generated_ids = outputs[:, inputs.shape[1]:]
    raw_output = qwen_tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()

    # 用正则表达式提取 ['food1','food2'] 格式的内容
    # 匹配单引号/双引号包裹的字符串，支持中英文食物名
    food_pattern = r"['\"]([^'\"]+)['\"]"
    food_list = re.findall(food_pattern, raw_output)

    # 5. 过滤空值（避免模型输出格式异常导致的空元素）
    food_list = [food.strip() for food in food_list if food.strip()]
    print(f"提取出的食物关键词{food_list}")

    return food_list


In [17]:
# 若检索不出用户提到的菜名，则让模型自己检索
def is_food_teach(query, qwen_model, qwen_tokenizer):
    """
    基于 QWen-7B 模型，判定是推荐食谱还是具体的食物制作
    :param query: 用户查询（字符串）
    :param qwen_model: 加载好的 QWen-7B 模型
    :param qwen_tokenizer: 加载好的 QWen 分词器
    :return:
    """
    core_query = extract_core_pair(query)
    # 1. 构造提示词模板（替换 {{query}} 为实际用户查询）
    prompt_template = f"""###任务：基于用户的询问，返回食谱信息
    用户查询（query）：{core_query}

    ###输出要求：
    1. 若需要的是你推荐美食，则输出YES
    2. 若需要的是你提供这道菜的食谱，则返回相应的做法
    3. 用英文回答
    """

    # 填充用户查询到提示词中
    #prompt = prompt_template.format(query=query)

    # 2. 用 QWen 分词器处理提示词（适配 QWen 聊天格式）
    inputs = qwen_tokenizer.apply_chat_template(
        [
            {'role':"system", "content": '你是一个专业的美食顾问，你需要根据用户询问，判定他是需要你推荐美食还是提供这道菜的食谱'},
            {"role": "user", "content": prompt_template}
        ],  # 按 QWen 要求的聊天格式组织
        add_generation_prompt=True,  # 自动添加模型回复的前缀
        return_tensors="pt"  # 返回 PyTorch 张量
    ).to(qwen_model.device)  # 移到模型所在设备（GPU/CPU）

    # 3. 模型生成结果（控制输出长度，避免冗余）
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = qwen_model.generate(
            inputs,
            max_new_tokens=4000,  # 食物列表长度有限，50个token足够
            temperature=0.1,  # 降低随机性，确保输出格式稳定
            top_p=0.9,
            do_sample=False,  # 确定性生成，避免重复结果
            eos_token_id=qwen_tokenizer.eod_id  # 遇到结束符停止生成
        )

    # 4. 解码输出并提取食物列表
    # 提取模型生成的部分（排除输入提示词）
    generated_ids = outputs[:, inputs.shape[1]:]
    raw_output = qwen_tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()


    return raw_output


In [18]:
query = 'Kung Pao chicken'
reslut = is_food_teach(query, qwen_model, qwen_tokenizer)
print(reslut)

/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/wby/anaconda3/envs/pytorch/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


YES
Here's a recipe for Kung Pao Chicken:
Ingredients:
- 500g boneless, skinless chicken breast, cut into small pieces
- 1/4 cup vegetable oil
- 1/4 cup roasted peanuts
- 1 red bell pepper, diced
- 1 green bell pepper, diced
- 1 onion, diced
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 tablespoon cornstarch
- 1 tablespoon soy sauce
- 1 tablespoon hoisin sauce
- 1 tablespoon rice vinegar
- 1 teaspoon sugar
- 1/4 teaspoon red pepper flakes
Instructions:
1. In a bowl, mix together the chicken pieces, cornstarch, and soy sauce.
2. Heat the oil in a wok or large skillet over high heat. Add the chicken and cook until browned on all sides, about 5 minutes.
3. Remove the chicken from the pan and set aside.
4. In the same pan, add the peanuts, red bell pepper, green bell pepper, onion, garlic, and ginger. Cook for 2-3 minutes, stirring occasionally.
5. Return the chicken to the pan and stir to combine with the vegetables.
6. In a small bowl, whisk together the hoisin sauce, rice 

In [19]:
# 2. 测试查询
query1 = 'i want to cook apple pie and orange'
result4 = is_food_list(query1, qwen_model, qwen_tokenizer)
print(f"查询：{query1}")
print(f"提取的食物列表：{result4}")  # 预期输出：['apple pie']

# 额外测试（验证多食物、非食物过滤场景）
query2 = 'i want to make yu xiang shredded pork and calorie < 30'
result5 = is_food_list(query2, qwen_model, qwen_tokenizer)
print(f"\n查询：{query2}")
print(f"提取的食物列表：{result5}")  # 预期输出：['orange', 'banana bread', 'milk']

query3 = 'i want to make apple pie and find USA great !'
result6 = is_food_list(query3, qwen_model, qwen_tokenizer)
print(f"\n查询：{query3}")
print(f"提取的食物列表：{result6}")  # 预期输出：['cake']


🎯 提取核心搭配：[i want to cook apple pie and orange] → [cook apple pie and orange]
提取出的食物关键词['apple pie', 'orange']
查询：i want to cook apple pie and orange
提取的食物列表：['apple pie', 'orange']
🎯 提取核心搭配：[i want to make yu xiang shredded pork and calorie < 30] → [make yu xiang shredded pork and]
提取出的食物关键词['yu xiang shredded pork']

查询：i want to make yu xiang shredded pork and calorie < 30
提取的食物列表：['yu xiang shredded pork']
🎯 提取核心搭配：[i want to make apple pie and find USA great !] → [make apple pie and find usa great]
提取出的食物关键词['apple pie']

查询：i want to make apple pie and find USA great !
提取的食物列表：['apple pie']


In [20]:
#过滤函数，过滤出提取的哪些单词是食物
def filter_list(result_list, qwen_model, qwen_tokenizer):
    print(result_list)
    final_results = []
    for result in result_list:
        if is_food_query_final_solution(result, qwen_model, qwen_tokenizer) == 'YES':
            final_results.append(result)
    return final_results

test_list = filter_list(result6, qwen_model, qwen_tokenizer)
print(test_list)

['apple pie']
🔍 是否是食物核心搭配判断：[how to make apple pie] → 输出：[YES]
['apple pie']


In [21]:
#提取口味
def extract_taste_words(query):
    # 1. 定义所有口味相关关键词（包含基础和“偏XX”表达）
    taste_keywords = [
        "sour", "tart", "slightly sour",  # 酸/偏酸
        "sweet", "sugary", "slightly sweet",  # 甜/偏甜
        "bitter", "slightly bitter",  # 苦/偏苦
        "spicy", "hot", "peppery", "slightly spicy", "mildly spicy",  # 辣/偏辣
        "salty", "slightly salty"  # 咸/偏咸
    ]

    # 2. 清理特殊符号，统一转为小写
    query_clean = re.sub(r'[?.,!;<>≤≥]', '', query).strip().lower()

    # 3. 构建正则模式：优先匹配多词短语（如 slightly sour），避免被拆分为单字
    # 按关键词长度（空格数）排序，多词在前，单词在后
    sorted_keywords = sorted(taste_keywords, key=lambda x: len(x.split()), reverse=True)
    # 转义特殊字符，用|拼接，添加单词边界\b确保精准匹配
    pattern = r'\b(' + '|'.join(re.escape(key) for key in sorted_keywords) + r')\b'

    # 4. 提取所有匹配的口味词，去重并保持出现顺序
    matched_tastes = re.findall(pattern, query_clean)
    # 去重（保留首次出现的词）
    seen = set()
    result = []
    for taste in matched_tastes:
        if taste not in seen:
            seen.add(taste)
            result.append(taste)

    return result

In [22]:
# 去重
def del_same(arrlist):
    seen = set()
    result = []
    for arr in arrlist:
        if arr not in seen:
            seen.add(arr)
            result.append(arr)
    return result

In [23]:
def parse_conditions_first(query):
    field_mapping = {
        "calorie": "Calories", "calories": "Calories", "kcal": "Calories",
        "time": "TotalTime", "totaltime": "TotalTime", "duration": "TotalTime",
        "serving": "RecipeServings", "servings": "RecipeServings", "portion": "RecipeServings"
    }
    operator_mapping = {
        "<": "<", "<=": "<=", ">": ">", ">=": ">=", "=": "=", "==": "=",
        'less than':"<", 'greater than':">", 'less than or equal to':"<=",
        'greater than or equal to':">=", 'equal to':"="
    }

    pattern = r"(\w+\s*\w*)\s*([<>=]{1,2}|\w+\s+\w+(\s+\w+)?)\s*(\d+\.?\d*)"  # 补充匹配英文运算符（如less than）
    matches = re.findall(pattern, query.lower())
    conditions = []

    for match in matches:
        # 修正：match结构因补充英文运算符变了，取第0个为关键词、第1个为运算符、第3个为数值
        user_filed, user_op, _, value = match
        user_filed_clean = user_filed.replace(" ", "")
        meta_field = None
        for key in field_mapping:
            if key in user_filed_clean:
                meta_field = field_mapping[key]
                break
        if not meta_field:
            continue
        # 处理英文运算符（如"less than"）
        chroma_op = operator_mapping.get(user_op.strip(), None)
        if not chroma_op:
            continue
        conditions.append([meta_field, chroma_op, str(value)])  # 每个条件都是列表

    if len(conditions) == 0:
        return None
    elif len(conditions) >= 1:
        # 修复：初始化合并列表，循环添加后续条件+and
        merged_conditions = conditions[0]  # 先拿第一个条件（列表）
        for cond in conditions[1:]:  # 遍历剩余条件
            merged_conditions += ['and'] + cond  # 列表拼接：[A]+[and]+[B]
        return merged_conditions

In [24]:
#存放RAG检索后的数组
rag_lists = []
rag_name_lists = []

In [25]:
def rag_recipe_qa_fixed(user_query, model, tokenizer, prompt_template, top_k=6):
    """
    修复后的完整RAG问答函数：解决解码空白问题
    """
    print(f"🔍 正在处理查询：{user_query}")
    # 1. 检索结果（不变）
    #提取一句话中食物的单词
    conditions  = parse_conditions_first(user_query)
    test_list = extract_taste_words(user_query)
    result_list = is_food_list(user_query, model, tokenizer)
    filtered_results = filter_list(result_list, qwen_model, tokenizer)
    filtered_results = del_same(test_list+filtered_results)
    print(f"过滤后的数组{filtered_results}")
    if conditions:
        querystr = ' '.join(filtered_results+['.Condition is']+ conditions)
    else:
        querystr = ' '.join(filtered_results)
    print(f"过滤后的结果{querystr}")
    retrieved_results = rag_retrieve(query=querystr, top_k=top_k)

    for i in range(len(retrieved_results)):
        rag_name_lists.append(retrieved_results[i]['meta']['Name'])
        rag_lists.append(retrieved_results[i])
    if not retrieved_results:
        return "抱歉，暂时没有找到符合您需求的美食"


    if len(filtered_results) == 0 and is_food_query_final_solution(user_query, model, qwen_tokenizer) != 'YES' and len(test_list) == 0:
        non_food_reply = "Sorry, I can only return data related to food and recipes, and cannot help with other types of queries."
        print(f"❌ 非食物查询，不执行检索 → 回复：{non_food_reply}")
        return non_food_reply


    # 2. 拼接提示词（不变）
    final_prompt = build_recipe_prompt(
        query=user_query,
        retrieved_results=retrieved_results,
        prompt_template=prompt_template
    )
    # 3. 模型生成：关键修改——增大max_new_tokens，关闭截断
    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(
            [   {'role':"system",'content':'你是一个专业的厨师，向用户推荐你的食谱, 并且你能详细的有逻辑的描述你的理由'},
                {"role": "user", "content": final_prompt}
            ],
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        # 关键修改：max_new_tokens设为800（足够生成完整推荐），加eos_token_id避免无意义生成
        outputs = model.generate(
            inputs,
            max_new_tokens=2000,  # 从500增至800，确保生成完整
            min_new_tokens=50,   # 强制生成至少50个token（避免太短）
            temperature=0.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 遇到结束符停止，避免冗余
            attention_mask=torch.ones_like(inputs)
        )

    # 修复解码：即使有特殊token，也先保留原始文本再过滤
    final_answer = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)


    print(f"\n🎉 最终过滤后回答：\n{final_answer}")
    print(type(final_answer))
    return final_answer if final_answer.strip() else "模型已生成内容，但需调整解码逻辑（见原始内容）"

In [26]:
prompt_template_final_fix = """
# 立即执行：基于检索结果推荐食谱（理由需自然改写,理由需包含检索数据中的 1 个具体食材或步骤。禁止生硬复制）
用户查询（query）：{{query}}
你的唯一任务：
1. 从下方3条检索结果中选distance≤0.4的推荐（都符合）；
2. 每个推荐必须包含：Doc ID、【食谱名（meta.Name）】、热量（meta.Calories）、总时间（meta.TotalTime）、口味(taste)；
3. 重点：推荐理由需满足以下2点（这是关键）：
   a. 信息来源：必须基于检索结果的text字段（如食材、步骤特点、口感相关描述），禁止编造任何未提及的内容（如没提“低糖”就不能说）；
   b. 语言创作：不能生硬复制text原文，要把text信息和用户需求（香蕉、时间<30分钟）结合，用自然的英文重新组织（比如把“premash the banana”改成“you can premash the banana in advance for easier mixing”）；
4. 最终回答用英文, 但Reason用中文回答，只输出推荐列表（至少1个最多3个,尽量3个，如果一个都没有则输出没有推荐的食谱），不输出模板文字或多余内容。
5. 请根据菜谱名来判断这道菜的口味如何(酸、甜、苦、辣、咸)



### 可用的RAG检索结果(若不符合用户提问的菜谱名(Name)，你可以自己编写)
{{retrieved_results}}

### 输出格式（理由必须自然改写，按这个结构来输出，不可省略任何参数）
请按照以下JSON格式输出(不可以遗漏任何一个花括号！)：
{
    "Doc ID": {doc_id},
    "Name": {Name},
    "Calories": {Calories},
    "Total Time": {TotalTime},
    "Reason": "你自己编写的详细且令人信服的理由可以结合时间(TotalTime)和卡路里(Calories)来进行说明(用中文回答)",
    "taste": "你自己判定的口味（用中文回答。口味有：酸、甜、苦、辣、咸、偏酸、偏甜、偏苦、偏辣、偏咸）"
 },
{
    "Doc ID": {doc_id},
    "Name": {Name},
    "Calories": {Calories},
    "Total Time": {TotalTime},
    "Reason": "你自己编写的详细且令人信服的理由可以结合时间(TotalTime)和卡路里(Calories)来进行说明(用中文回答)",
    "taste": "你自己判定的口味（用中文回答。口味有：酸、甜、苦、辣、咸、偏酸、偏甜、偏苦、偏辣、偏咸）"
}
"""

In [47]:
#测试版
prompt_template_final_fix2 = """
# 立即执行：基于检索结果推荐食谱（理由需自然改写,理由需包含检索数据中的 1 个具体食材或步骤。禁止生硬复制）
用户查询（query）：{{query}}
你的唯一任务：
1. 从下方3条检索结果中选distance≤0.4的推荐（都符合）；
2. 每个推荐必须包含：【食谱名（meta.Name）】、热量（meta.Calories）、总时间（meta.TotalTime）、口味(taste)；
3. 重点：推荐理由需满足以下2点（这是关键）：
   a. 信息来源：基于检索结果的text字段（如食材、步骤特点、口感相关描述），禁止编造任何未提及的内容（如没提“低糖”就不能说）；
   b. 语言创作：不能生硬复制text原文，要把text信息和用户需求（香蕉、时间<30分钟）结合，用自然的英文重新组织（比如把“premash the banana”改成“you can premash the banana in advance for easier mixing”）；
4. 最终回答用英文, 但Reason用中文回答，只输出推荐列表（至少1个最多3个,尽量3个，如果一个都没有则输出没有推荐的食谱），不输出模板文字或多余内容。
5. 请根据菜谱名来判断这道菜的口味如何(sour, sweet, bitter, spicy, salty)
6. RAG检索结果只是参考，根据实际需要进行参考然后回答问题，不一定非要参照RAG结果



### 可用的RAG检索结果(若不符合用户提问的菜谱名(Name)，你可以自己编写)
{{retrieved_results}}

### 输出格式（理由必须自然改写，按这个结构来输出，不可省略任何参数）
请按照以下JSON格式输出(不可以遗漏任何一个花括号！不要遗漏任何一个花括号！)：
[
{
    "Name": {Name},
    "Calories": {Calories},
    "Total Time": {TotalTime},
    "Reason": "你自己编写的详细且令人信服的理由可以结合时间(TotalTime)和卡路里(Calories)来进行说明",
    "taste": "你自己判定的口味（口味有：sour, sweet, bitter, spicy, salty, slightly sour, slightly sweet, slightly bitter, slightly spicy, slightly salty）",
 }
]
"""

In [50]:
user_query = "how to make fish"
rag_lists = []
rag_name_lists = []

# 2. 调用完整RAG函数生成回答
# （model、tokenizer、prompt_template都是你之前加载/定义好的，直接用）
"""final_answer = rag_recipe_qa(
    user_query=user_query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
    prompt_template=prompt_template,
    top_k=3
)"""
final_answer = rag_recipe_qa_fixed(
    user_query=user_query,
    model=qwen_model,
    tokenizer=qwen_tokenizer,
    prompt_template=prompt_template_final_fix2,
    top_k=3
)


🔍 正在处理查询：how to make fish
🎯 提取核心搭配：[how to make fish] → [make fish]
提取出的食物关键词['fish']
['fish']
🔍 是否是食物核心搭配判断：[eat fish] → 输出：[YES]
过滤后的数组['fish']
过滤后的结果fish

🎉 最终过滤后回答：
 [
            {
               "Name": "Spanish-Style Fish",
               "Calories": 197.2,
               "Total Time": 50.0,
               "Reason": "This recipe is a great option if you're looking for a healthy and flavorful way to prepare fish. The saffron and cayenne give it a unique taste that's sure to impress your guests. It also cooks relatively quickly at only 50 minutes, making it perfect for a busy weeknight.",
               "taste": "spicy"
            },
            {
               "Name": "Bake Fish",
               "Calories": 35.2,
               "Total Time": 195.0,
               "Reason": "If you're looking for a simple and easy-to-make fish recipe, this one is a great choice. With only a few ingredients and minimal preparation time, it's perfect for a quick weeknight dinner. The red chilies a

In [45]:
import re
import json

def fix_recipe_json(raw_str: str):
    """终极必成：模糊匹配字段，强制提取并重构JSON"""
    # 步骤1：超级清理，只保留有用内容（去掉所有特殊符号，只留字母、数字、中文、空格、逗号、冒号、花括号）
    cleaned = raw_str.strip()
    # 只保留核心字符，彻底消除乱码、特殊符号干扰
    cleaned = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s,:{}]', '', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned)  # 合并多余空格

    # 步骤2：提取所有食谱块（只要是花括号包着的内容都算）
    recipe_blocks = re.findall(r'\{([^}]+)\}', cleaned)  # 匹配 { 和 } 之间的所有内容
    if not recipe_blocks:
        print("未找到食谱块，尝试直接提取文本中的字段...")
        # 如果没有花括号，直接把整个清理后的内容当一个食谱块处理
        recipe_blocks = [cleaned]

    final_recipes = []
    for block in recipe_blocks:
        # 步骤3：模糊匹配字段（不区分大小写、允许字段名前后有乱码）
        recipe = {}
        # 匹配Name（支持 "Name:"、"name:"、" Name :" 等各种格式）
        name = re.search(r'name\s*:\s*([^,}{]+)', block, re.IGNORECASE)
        if name:
            recipe['Name'] = name.group(1).strip()

        # 匹配Calories
        calories = re.search(r'calories\s*:\s*([^,}{]+)', block, re.IGNORECASE)
        if calories:
            recipe['Calories'] = calories.group(1).strip()

        # 匹配Total Time（支持 "Total Time"、"total time"、"Totaltime"）
        total_time = re.search(r'total\s*time\s*:\s*([^,}{]+)', block, re.IGNORECASE)
        if not total_time:
            total_time = re.search(r'totaltime\s*:\s*([^,}{]+)', block, re.IGNORECASE)
        if total_time:
            recipe['Total Time'] = total_time.group(1).strip()

        # 匹配Reason（从Reason: 到 taste: 或 结束 之间的内容）
        reason = re.search(r'reason\s*:\s*([^t}{]+)', block, re.IGNORECASE)
        if reason:
            recipe['Reason'] = reason.group(1).strip().rstrip(',')

        # 匹配taste
        taste = re.search(r'taste\s*:\s*([^,}{]+)', block, re.IGNORECASE)
        if taste:
            recipe['taste'] = taste.group(1).strip().rstrip(',')

        # 步骤4：只要有Name就保留（避免空字典）
        if recipe.get('Name'):
            final_recipes.append(recipe)

    # 步骤5：生成标准JSON
    standard_json = json.dumps(final_recipes, ensure_ascii=False, indent=2)
    print("重构后的标准JSON：")
    print(standard_json)
    return final_recipes if final_recipes else "未提取到有效食谱"

In [51]:


#答案转为json
#answer_json_str = f"[{final_answer}]"
#answer_json_array = fix_recipe_json(final_answer)
test = json.loads(final_answer)



#print(answer_json_array)
print(test)


[{'Name': 'Spanish-Style Fish', 'Calories': 197.2, 'Total Time': 50.0, 'Reason': "This recipe is a great option if you're looking for a healthy and flavorful way to prepare fish. The saffron and cayenne give it a unique taste that's sure to impress your guests. It also cooks relatively quickly at only 50 minutes, making it perfect for a busy weeknight.", 'taste': 'spicy'}, {'Name': 'Bake Fish', 'Calories': 35.2, 'Total Time': 195.0, 'Reason': "If you're looking for a simple and easy-to-make fish recipe, this one is a great choice. With only a few ingredients and minimal preparation time, it's perfect for a quick weeknight dinner. The red chilies and ginger add a nice kick of flavor, while the coriander leaves provide a refreshing touch.", 'taste': 'spicy'}, {'Name': 'Italian Baked Fish', 'Calories': 55.6, 'Total Time': 40.0, 'Reason': "This recipe is a great option if you're looking for a delicious and healthy way to enjoy fish. With only a few ingredients and minimal preparation time,

In [52]:
for rag in rag_lists:
    print(f"食物信息： {rag['text']}")

食物信息： RecipeId:171401
Name:Spanish-Style Fish
TotalTime:50.0
RecipeCategory:Spanish
Calories:197.2
RecipeServings:4.0
Ingredients and Quantities:
cod fish fillets:1 1/2
olive oil:1 -2
onion:1
shallots:2
garlic clove:1
tomatoes:1
cayenne pepper:1
saffron thread:1
Cooking steps:
1. Cut the fish into semi-small pieces; sauté the fish in heated olive oil until golden-brown; stir in onions, shallots, garlic, tomatoes, cayenne, saffron, and salt.
2. Simmer, uncovered, for about 40 minutes; season with fresh-ground pepper and a small handful of  tarragon just before serving.
食物信息： RecipeId:12835
Name:Bake Fish
TotalTime:195.0
RecipeCategory:Bass
Calories:35.2
RecipeServings:4.0
Ingredients and Quantities:
red chilies:1
ginger:3
coriander leaves:3
sugar:3
lime juice:2
tomatoes:1
spring onion:1
Cooking steps:
1. Wash slit lenghtwise and pat dry fish on both sides.
2. Marinate the fish with all the ingredients for 2 hours.
3. Using combination 4,put to roast for about 30 minutes or until the fis

In [ ]:
#查看模型层的参数名
for name, param in qwen_model.named_parameters():
    print(name)

In [ ]:
#如果检索结果用户的选择不在检索结果里，则让大模型自己去生成食物制作
if not(answer_json_array[0]['Name'] in rag_name_lists):
    final_reslut = is_food_teach(answer_json_array[0]['Name'], qwen_model, qwen_tokenizer)
    print(final_reslut)

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    TrainingArguments, DataCollatorForLanguageModeling, Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
json_data_path = '/home/wby/projects/week5/data/food_task.json'
output_dir = '/home/wby/projects/week5/data/qwen_food_lora' #存放QLORA适配器的地方

# 1. 加载数据集（不用改）
print("正在加载数据集...")
dataset = load_dataset("json", data_files=json_data_path)
train_dataset = dataset["train"]  # 新手不用分验证集，全用训练
print(f"数据集加载完成，共{len(train_dataset)}条数据")
print(dataset['train'])

qwen_tokenizer.pad_token = qwen_tokenizer.eos_token